<a href="https://colab.research.google.com/github/nmansour67/skills-introduction-to-github/blob/main/Yield_Engine_Protocol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# SECTION 4: THE YIELD ENGINE SIMULATOR
# ==========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)
days_to_simulate = 365
slots_per_day = 20

# FINANCIAL CONSTANTS
REVENUE_PER_SCAN = 1000
PENALTY_CLASH = 500  # Cost of apology/overtime

# --- STEP 1: DEFINE THE STRATEGIES ---

def run_simulation(strategy_name):
    total_revenue = 0
    total_clashes = 0

    for day in range(days_to_simulate):
        # 1. Generate Demand (Patients)
        # We assume we have unlimited demand, but we can only schedule X slots.
        # Each patient has a "Show Probability" predicted by AI.
        # Most represent reliable patients (90%), some are flaky (40%).
        patient_pool_probs = np.random.beta(a=2, b=1, size=50) # Skewed towards high prob

        scheduled_patients = []

        # --- STRATEGY LOGIC ---
        if strategy_name == "The Coward (Standard)":
            # Just book 20 slots. Stop.
            probs = patient_pool_probs[:slots_per_day]
            scheduled_patients = [1 if np.random.random() < p else 0 for p in probs]

        elif strategy_name == "The Gambler (Blind Overbook)":
            # Book 24 slots (20% overbook) regardless of risk
            probs = patient_pool_probs[:24]
            scheduled_patients = [1 if np.random.random() < p else 0 for p in probs]

        elif strategy_name == "The AI Sniper (Smart Overbook)":
            # Book 20 slots.
            # THEN, scan the list. If prob < 0.6, add a "Double Book" to that slot.
            current_bookings = []
            probs = patient_pool_probs[:30] # Look at waiting list

            slot_count = 0
            for p in probs:
                if slot_count < slots_per_day:
                    # Book the slot
                    current_bookings.append(p)
                    # AI DECISION: Is this person risky?
                    if p < 0.60:
                        # YES! Double book this specific slot immediately
                        current_bookings.append(probs[slot_count + 1]) # Add another
                    slot_count += 1

            # Simulate who actually showed up
            scheduled_patients = [1 if np.random.random() < p else 0 for p in current_bookings]

        # --- STEP 2: CALCULATE DAILY OUTCOME ---
        shows = sum(scheduled_patients)

        # Revenue is capped at capacity (can't scan more than 20)
        scans_completed = min(shows, slots_per_day)
        daily_revenue = scans_completed * REVENUE_PER_SCAN

        # Did we clash? (More people showed up than slots)
        daily_clashes = max(0, shows - slots_per_day)
        daily_penalty = daily_clashes * PENALTY_CLASH

        total_revenue += (daily_revenue - daily_penalty)
        total_clashes += daily_clashes

    return total_revenue, total_clashes

# --- RUN THE SIMULATIONS ---
rev_coward, clash_coward = run_simulation("The Coward (Standard)")
rev_gambler, clash_gambler = run_simulation("The Gambler (Blind Overbook)")
rev_ai, clash_ai = run_simulation("The AI Sniper (Smart Overbook)")

# --- STEP 3: THE BOARDROOM REVEAL ---
print("-" * 50)
print("ANNUAL PERFORMANCE REPORT (365 Days)")
print("-" * 50)
print(f"1. THE COWARD (No Overbooking):")
print(f"   Revenue: ${rev_coward:,.0f}")
print(f"   Clashes: {clash_coward} (Peaceful, but poor)")
print("-" * 50)
print(f"2. THE GAMBLER (Blind Overbooking):")
print(f"   Revenue: ${rev_gambler:,.0f}")
print(f"   Clashes: {clash_gambler} (Chaos! Staff is quitting!)")
print("-" * 50)
print(f"3. THE AI SNIPER (Targeted Overbooking):")
print(f"   Revenue: ${rev_ai:,.0f}")
print(f"   Clashes: {clash_ai} (Manageable)")
print("-" * 50)

advantage = rev_ai - rev_coward
print(f"🚀 AI PROFIT ADVANTAGE: ${advantage:,.0f} PER YEAR")

--------------------------------------------------
ANNUAL PERFORMANCE REPORT (365 Days)
--------------------------------------------------
1. THE COWARD (No Overbooking):
   Revenue: $4,863,000
   Clashes: 0 (Peaceful, but poor)
--------------------------------------------------
2. THE GAMBLER (Blind Overbooking):
   Revenue: $5,827,000
   Clashes: 8 (Chaos! Staff is quitting!)
--------------------------------------------------
3. THE AI SNIPER (Targeted Overbooking):
   Revenue: $6,506,500
   Clashes: 83 (Manageable)
--------------------------------------------------
🚀 AI PROFIT ADVANTAGE: $1,643,500 PER YEAR
